# SQL Asset - Partitioned

## Basic setup

Imports

In [ ]:
import great_expectations as gx
import great_expectations.expectations as gxe

from great_expectations.checkpoint.v1_checkpoint import Checkpoint
from great_expectations.core.validation_definition import ValidationDefinition
from great_expectations.exceptions import DataContextError

from constants import DB_CONNECTION_STRING, TABLE_NAME

Define our names

In [ ]:
asset_name = "trip-data"
batch_definition_name = "partition-by-month"
checkpoint_name = "my_checkpoint"
datasource_name = "postgres"
suite_name = "passenger_count_checker"

In [ ]:
context = gx.get_context(mode="file")

try:
    datasource = context.sources.add_postgres(datasource_name, connection_string=DB_CONNECTION_STRING)
    data_asset = datasource.add_table_asset(name=asset_name, table_name=TABLE_NAME)
    batch_def = data_asset.add_batch_definition_monthly(batch_definition_name, "tpep_pickup_datetime", sort_asc=False)
    suite = context.add_expectation_suite(suite_name)
    suite.add_expectation(
        gxe.ExpectColumnValuesToBeBetween(
            column="passenger_count", min_value=0, max_value=10
        )
    )
    print("Created entities")
except DataContextError:
    print("Entities already exist")


## Check the batches for our config

In [ ]:
batch_request = batch_def.build_batch_request()
batches = data_asset.get_batch_list_from_batch_request(batch_request)

# Verify that we have multiple batches in the correct order
print([b.id for b in batches])

## Run ValidationDefinition

In [ ]:
validation_definition = ValidationDefinition(name="epheemeral validation definition", data=batch_def, suite=suite)

results = validation_definition.run()
print(results)

## Create and run a Checkpoint

In [ ]:
try:
    checkpoint = context.checkpoints.add(Checkpoint(
        name=checkpoint_name,
        validation_definitions=[
            ValidationDefinition(name="validation_def", data=batch_def, suite=suite)
        ],
        actions=[],
    ))
except DataContextError:
    # pass; the datasource, data_asset, batch_def, and suite already exist
    ...

results = checkpoint.run()
print(results)